# CLASE 2 - Validaciones, respuestas tipadas y estructuras con Pydantic
---

## 1. Introducción a Pydantic y BaseModel (30 min)

### Qué es Pydantic

**Pydantic** es una biblioteca de Python ampliamente utilizada para la **validación de datos** y su **conversión (parsing)** utilizando clases basadas en anotaciones de tipo. Fue diseñada para facilitar el manejo de datos estructurados, especialmente en contextos como APIs, aplicaciones web, procesamiento de entradas de usuarios o integración con bases de datos.

El núcleo de Pydantic se basa en el uso de clases que heredan de `BaseModel`, una clase proporcionada por la biblioteca. Al definir una clase que hereda de `BaseModel`, se pueden especificar atributos junto con sus tipos, y Pydantic se encargará de:


* Validar automáticamente los datos entrantes, asegurando que coincidan con los tipos esperados.
* Convertir (parsear) automáticamente los datos a los tipos definidos cuando sea posible.
* Generar errores claros y estructurados en caso de datos inválidos.
* Definición clara de estructuras de datos.
* Serialización/deserialización fácil.
* Compatible con FastAPI.
* Ofrecer métodos útiles como `.dict()`, `.json()` o `.schema()` para exportar y manipular los modelos fácilmente.


Gracias a su integración con la tipificación estática de Python (tipo `int`, `str`, `float`, `datetime`, `List`, `Dict`, etc.), Pydantic promueve un código más **legible**, **seguro** y **autodocumentado**. Es especialmente popular en frameworks modernos como **FastAPI**, donde se utiliza para definir modelos de entrada y salida en endpoints RESTful, garantizando la integridad de los datos en todo momento.




---

## Para saber si **Pydantic es compatible con FastAPI**, debes considerar la **versión de FastAPI** que estás utilizando, ya que actualmente:

---


###  Compatibilidad por versiones

| FastAPI           | Compatible con    | Notas                                                                             |
| ----------------- | ----------------- | --------------------------------------------------------------------------------- |
| FastAPI ≤ 0.103.x | **Pydantic v1.x** | Usada en la mayoría de los proyectos en producción                                |
| FastAPI ≥ 0.104.x | **Pydantic v2.x** | Soporte oficial para Pydantic 2, aún en proceso de madurez en algunas extensiones |

---

###  Cómo verificar qué versión de FastAPI y Pydantic tienes instalada

Abre tu terminal y ejecuta:

```bash
pip show fastapi
pip show pydantic
```

Esto te mostrará algo como:

```
Name: fastapi
Version: 0.103.2
...
Name: pydantic
Version: 1.10.13
...
```

---

###  ¿Qué hacer si tienes FastAPI y quieres usar Pydantic 2?

Si estás comenzando un proyecto nuevo y deseas usar **Pydantic v2**, asegúrate de instalar una versión de FastAPI que sea **0.104.0 o superior**, por ejemplo:

```bash
pip install "fastapi>=0.104.0" "pydantic>=2.0"
```

Pero si estás trabajando en un proyecto existente con FastAPI v0.103.x o menor, **quédate con Pydantic v1.10.x**, así:

```bash
pip install "pydantic<2.0"
```

---


## Instalar Pydantic: la versión 1.x (la más usada y compatible con FastAPI hasta ahora):

```bash
pip install pydantic
```

### Instalar Pydantic: la versión 2.x (más reciente y con mejoras de rendimiento y nuevas características):

```bash
pip install "pydantic>=2.0"
```

> ⚠️ Nota: Si estás utilizando un framework como **FastAPI**, asegúrate de que la versión de Pydantic sea compatible con el framework, ya que muchas herramientas aún trabajan con Pydantic v1.x.



### Sintaxis básica

In [ ]:
from pydantic import BaseModel

class Usuario(BaseModel):
    nombre: str
    edad: int

usuario = Usuario(nombre="Ana", edad="25")  # edad se convierte automáticamente
print(usuario)


---

##  2. Modelos de Entrada y Salida (30 min)

###  Entrada vs. Salida

Separar los modelos de entrada (Request) y salida (Response) permite controlar lo que el cliente envía y recibe, evitando exponer campos sensibles.

###  Ejemplo:


In [ ]:
from pydantic import BaseModel

class TareaEntrada(BaseModel):
    titulo: str
    descripcion: str

class TareaSalida(BaseModel):
    id: int
    titulo: str
    descripcion: str
    completada: bool


####  Ejercicio:

Define modelos para una API de productos:



In [ ]:
from pydantic import BaseModel

class ProductoEntrada(BaseModel):
    nombre: str
    precio: float

class ProductoSalida(BaseModel):
    id: int
    nombre: str
    precio: float
    disponible: bool


---

## 3. Validaciones automáticas: tipos, longitudes, rangos (45 min)

###  Tipos y restricciones

Usa `Field` para establecer validaciones sobre los atributos:



In [ ]:
from pydantic import BaseModel, Field

class Usuario(BaseModel):
    username: str = Field(..., min_length=4, max_length=12)
    edad: int = Field(..., gt=17, lt=100)


###  Campos comunes:

* `min_length`, `max_length`: longitud de strings
* `gt`, `lt`, `ge`, `le`: mayor/menor que
* `regex`: expresiones regulares para validar formatos

###  Ejercicio:

Crear un modelo para una reserva:



In [ ]:
class Reserva(BaseModel):
    cliente: str = Field(..., min_length=3)
    fecha: str = Field(..., regex=r"\\d{4}-\\d{2}-\\d{2}")
    personas: int = Field(..., ge=1, le=10)


---

##  4. Respuestas personalizadas con JSONResponse (30 min)

###  Devolver respuestas limpias



In [ ]:
from fastapi import FastAPI, APIRouter, HTTPException, status
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

# =====================================================
# MODELOS Pydantic
# =====================================================

class RecursoBase(BaseModel):
    """Modelo base para los recursos"""
    nombre: str = Field(..., min_length=3, max_length=50, description="Nombre del recurso", examples=["Mi Recurso"])
    descripción: Optional[str] = Field(None, max_length=200, description="Descripción opcional del recurso", examples=["Esta es una descripción de ejemplo"])


class RecursoCreate(RecursoBase):
    """Modelo para crear un recurso (entrada)"""
    pass


class RecursoUpdate(BaseModel):
    """Modelo para actualizar parcialmente un recurso"""
    nombre: Optional[str] = Field(None, min_length=3, max_length=50, examples=["Recurso Actualizado"])
    descripción: Optional[str] = Field(None, max_length=200, examples=["Nueva descripción"])


class RecursoResponse(RecursoBase):
    """Modelo de respuesta para un recurso"""
    item_id: int = Field(..., gt=0, description="ID único del recurso")


class UsuarioBase(BaseModel):
    """Modelo base para usuarios"""
    username: str = Field(..., min_length=4, max_length=16, description="Nombre de usuario", examples=["juan123"])
    email: str = Field(
        ...,
        pattern=r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$",
        description="Correo electrónico válido",
        examples=["usuario@ejemplo.com"]
    )
    edad: int = Field(..., gt=0, lt=120, description="Edad entre 1 y 119", examples=[25])


class UsuarioCreate(UsuarioBase):
    """Modelo para crear usuario"""
    pass


class UsuarioUpdate(BaseModel):
    """Modelo para actualizar parcialmente un usuario"""
    username: Optional[str] = Field(None, min_length=4, max_length=16, examples=["nuevo_usuario"])
    email: Optional[str] = Field(None, pattern=r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$", examples=["nuevo@ejemplo.com"])
    edad: Optional[int] = Field(None, gt=0, lt=120, examples=[30])


class UsuarioResponse(UsuarioBase):
    """Modelo de respuesta para usuario"""
    user_id: int = Field(..., gt=0, description="ID único del usuario")


# =====================================================
# "BASE DE DATOS" EN MEMORIA
# =====================================================

db_recursos: List[Dict] = []
next_recurso_id = 1

db_usuarios: List[Dict] = []
next_usuario_id = 1

# =====================================================
# ROUTERS PERSONALIZADOS
# =====================================================

# Router de Recursos
recursos_router = APIRouter(
    prefix="/recursos",
    tags=["Recursos"],
    responses={404: {"description": "Recurso no encontrado"}}
)

# Router de Usuarios
usuarios_router = APIRouter(
    prefix="/usuarios",
    tags=["Usuarios"],
    responses={404: {"description": "Usuario no encontrado"}}
)

# -------------------
# Endpoints de Recursos
# -------------------

@recursos_router.post(
    "/",
    response_model=RecursoResponse,
    status_code=status.HTTP_201_CREATED,
    summary="Crear un nuevo recurso"
)
async def create_recurso(recurso: RecursoCreate):
    global next_recurso_id
    new_item = recurso.model_dump()
    new_item["item_id"] = next_recurso_id
    db_recursos.append(new_item)
    next_recurso_id += 1
    return new_item


@recursos_router.get(
    "/",
    response_model=List[RecursoResponse],
    summary="Obtener todos los recursos"
)
async def get_all_recursos():
    return db_recursos


@recursos_router.get(
    "/{item_id}",
    response_model=RecursoResponse,
    summary="Obtener un recurso por ID"
)
async def read_recurso(item_id: int):
    if item_id <= 0:
        raise HTTPException(status_code=422, detail="ID de recurso inválido.")
    for item in db_recursos:
        if item["item_id"] == item_id:
            return item
    raise HTTPException(status_code=404, detail="Recurso no encontrado")


@recursos_router.put(
    "/{item_id}",
    response_model=RecursoResponse,
    summary="Actualizar un recurso existente"
)
async def update_recurso(item_id: int, recurso: RecursoUpdate):
    for item in db_recursos:
        if item["item_id"] == item_id:
            update_data = recurso.model_dump(exclude_unset=True)
            item.update(update_data)
            return item
    raise HTTPException(status_code=404, detail="Recurso no encontrado")


@recursos_router.delete(
    "/{item_id}",
    status_code=status.HTTP_200_OK,
    summary="Eliminar un recurso"
)
async def delete_recurso(item_id: int):
    global db_recursos
    for item in db_recursos:
        if item["item_id"] == item_id:
            db_recursos.remove(item)
            return {"mensaje": "Recurso eliminado con éxito"}
    raise HTTPException(status_code=404, detail="Recurso no encontrado")


# -------------------
# Endpoints de Usuarios
# -------------------

@usuarios_router.post(
    "/",
    response_model=UsuarioResponse,
    status_code=status.HTTP_201_CREATED,
    summary="Crear un nuevo usuario"
)
async def create_usuario(usuario: UsuarioCreate):
    global next_usuario_id
    new_user = usuario.model_dump()
    new_user["user_id"] = next_usuario_id
    db_usuarios.append(new_user)
    next_usuario_id += 1
    return new_user


@usuarios_router.get(
    "/",
    response_model=List[UsuarioResponse],
    summary="Obtener todos los usuarios"
)
async def get_all_usuarios():
    return db_usuarios


@usuarios_router.get(
    "/{user_id}",
    response_model=UsuarioResponse,
    summary="Obtener un usuario por ID"
)
async def read_usuario(user_id: int):
    if user_id <= 0:
        raise HTTPException(status_code=422, detail="ID inválido.")
    for user in db_usuarios:
        if user["user_id"] == user_id:
            return user
    raise HTTPException(status_code=404, detail="Usuario no encontrado")


@usuarios_router.put(
    "/{user_id}",
    response_model=UsuarioResponse,
    summary="Actualizar un usuario"
)
async def update_usuario(user_id: int, usuario: UsuarioUpdate):
    for user in db_usuarios:
        if user["user_id"] == user_id:
            update_data = usuario.model_dump(exclude_unset=True)
            user.update(update_data)
            return user
    raise HTTPException(status_code=404, detail="Usuario no encontrado")


@usuarios_router.delete(
    "/{user_id}",
    status_code=status.HTTP_200_OK,
    summary="Eliminar un usuario"
)
async def delete_usuario(user_id: int):
    global db_usuarios
    for user in db_usuarios:
        if user["user_id"] == user_id:
            db_usuarios.remove(user)
            return {"mensaje": "Usuario eliminado con éxito"}
    raise HTTPException(status_code=404, detail="Usuario no encontrado")


# =====================================================
# APP PRINCIPAL
# =====================================================

app = FastAPI(
    title="API de Recursos y Usuarios",
    description="Ejemplo de API con FastAPI, APIRouter personalizado, validación con Pydantic y base en memoria.",
    version="2.1.0"
)

# Endpoint raíz
@app.get("/", summary="Página de inicio de la API")
async def root():
    return {
        "mensaje": "¡Bienvenido a la API de Recursos y Usuarios!",
        "version": "1.0.0",
        "documentacion": "/docs",
        "endpoints": {
            "recursos": "/recursos",
            "usuarios": "/usuarios"
        }
    }

# Incluir routers personalizados
app.include_router(recursos_router)
app.include_router(usuarios_router)

# =====================================================
# PUNTO DE ENTRADA
# =====================================================

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(
        "main:app",
        host="127.0.0.1",
        port=8000,
        reload=True,
        log_level="info"
    )


###  Personalización de errores:



In [ ]:
from fastapi import HTTPException

@app.get("/producto/{id}")
def obtener_producto(id: int):
    if id != 1:
        raise HTTPException(status_code=404, detail="Producto no encontrado")
    return {"id": id, "nombre": "Monitor"}


---

##  5. Integración con FastAPI: API estructurada (45 min)

###  Estructura recomendada:

```
api/
 ├── main.py
 ├── models.py
 └── routers/
      └── tareas.py
```

###  models.py



In [ ]:
from pydantic import BaseModel, Field

class TareaEntrada(BaseModel):
    titulo: str = Field(..., min_length=3)
    descripcion: str = Field(..., min_length=10)

class TareaSalida(BaseModel):
    id: int
    titulo: str
    descripcion: str
    completada: bool


###  main.py

In [ ]:
from fastapi import FastAPI
from routers import tareas

app = FastAPI(title="API de Tareas")

# Incluir rutas
app.include_router(tareas.router)

@app.get("/")
def leer_raiz():
    return {"mensaje": "Bienvenido a la API de Tareas. Visita /docs para ver la documentación."}


### routers/tareas.py

In [ ]:
from fastapi import APIRouter
from models import TareaEntrada, TareaSalida

router = APIRouter()

# Simulación de base de datos en memoria
tareas_db = []
id_actual = 1

@router.post("/tareas", response_model=TareaSalida)
def crear_tarea(tarea: TareaEntrada):
    global id_actual
    nueva_tarea = TareaSalida(
        id=id_actual,
        titulo=tarea.titulo,
        descripcion=tarea.descripcion,
        completada=False
    )
    tareas_db.append(nueva_tarea)
    id_actual += 1
    return nueva_tarea

@router.get("/tareas", response_model=list[TareaSalida])
def listar_tareas():
    return tareas_db


##  ¿Por qué usar `requirements.txt`?

* Facilita la instalación de dependencias con un solo comando.
* Garantiza que todos usen las **mismas versiones** de paquetes.
* Es útil para despliegues, entornos virtuales o contenedores Docker.

---

##  ¿Qué debe incluir `requirements.txt` para este proyecto?

Como tu proyecto usa **FastAPI** y **Uvicorn** (el servidor para ejecutar la API), necesitas incluir al menos:

```
fastapi
uvicorn[standard]
```

> `uvicorn[standard]` incluye soporte para recarga automática (`--reload`), logs y otras utilidades útiles para desarrollo.

---

##  Cómo crear e instalar desde `requirements.txt`

1. **Crear el archivo**:

   Crea un archivo llamado `requirements.txt` en la raíz del proyecto (por ejemplo, dentro de la carpeta `api/`).

   Contenido:

   ```txt
   fastapi
   uvicorn[standard]
   ```

2. **Instalar dependencias**:

   En tu terminal, navega a la carpeta del proyecto y ejecuta:

   ```bash
   pip install -r requirements.txt
   ```

   Esto instalará automáticamente todas las librerías listadas.

---

##  Verifica que estén instaladas:

Después de la instalación, puedes verificar que están disponibles:

```bash
pip show fastapi
pip show uvicorn
```

---



##  Ejecutar el servidor

Desde el directorio `api/`, ejecuta:

```bash
uvicorn main:app --reload
```

Y abre en tu navegador:

* Documentación interactiva: [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs)

---


##  6. Cursor IA como asistente (15 min)

###  Funciones:

* Generar modelos a partir de JSON.
* Sugerir validaciones.
* Detectar campos no tipados.

###  Actividad:

1. Pega un JSON en Cursor IA.
2. Solicita el modelo `BaseModel` con validaciones.
3. Refactoriza el modelo generado para entrada y salida.

---


## 📌 Proyecto Empresarial FastAPI – API de Soporte Técnico (CRUD completo)

### 🎯 Objetivo

Desarrollar una API REST empresarial para gestionar clientes y tickets de soporte técnico:

* ✅ Validación de datos con **Pydantic**.
* ✅ Estructura RESTful profesional.
* ✅ CRUD completo: crear, consultar, listar, actualizar y eliminar.
* ✅ Estructura modular.
* ✅ Código claro, seguro y extensible.

---


## 📁 Estructura del proyecto

```
crm_api/
├── main.py
├── models.py
├── requirements.txt
└── routers/
    ├── clientes.py
    └── tickets.py
```

---


#### 1. **Instalar el paquete validación de correos electrónicos a través de `pydantic`** llamado `email-validator`, que es requerido explícitamente cuando defines un campo de tipo `EmailStr` en un modelo de `Pydantic`

Abre tu terminal o consola (puedes usar la terminal de VS Code o Anaconda Prompt) y ejecuta:

```bash
pip install pydantic[email]
```

Esto instalará `email-validator`, que es un requerimiento opcional de `Pydantic` cuando usas `EmailStr`.

---


### `models.py` – Modelos Pydantic

In [ ]:
from pydantic import BaseModel, Field, EmailStr  # Importamos herramientas para validación

# Modelo de entrada para un cliente (lo que el usuario envía)
class ClienteEntrada(BaseModel):
    nombre: str = Field(..., min_length=3, example="Ana Gómez")  # Campo obligatorio con mínimo 3 caracteres
    email: EmailStr = Field(..., example="ana@empresa.com")      # Valida que sea un email válido

# Modelo de salida para un cliente (lo que responde la API)
class ClienteSalida(ClienteEntrada):
    id: int  # Incluye un campo adicional "id"

# Modelo de entrada para un ticket de soporte
class TicketEntrada(BaseModel):
    asunto: str = Field(..., min_length=5)         # Campo obligatorio con mínimo 5 caracteres
    descripcion: str = Field(..., min_length=10)   # Campo obligatorio con mínimo 10 caracteres
    cliente_id: int = Field(..., gt=0)             # El ID debe ser mayor que 0

# Modelo de salida para un ticket (incluye estado e ID)
class TicketSalida(TicketEntrada):
    id: int                 # ID del ticket
    estado: str             # Estado del ticket (ej: "Abierto", "Cerrado")


---

###  `routers/clientes.py` – Lógica de clientes


In [ ]:
from fastapi import APIRouter, HTTPException       # APIRouter para modularizar rutas, HTTPException para manejar errores
from models import ClienteEntrada, ClienteSalida   # Importamos nuestros modelos

router = APIRouter()           # Creamos un router para agrupar rutas de cliente
clientes_db = []               # Base de datos simulada en memoria (lista de clientes)
id_counter = 1                 # Contador simple para simular IDs autoincrementales

# Ruta para crear un cliente
@router.post("/clientes", response_model=ClienteSalida, status_code=201)
def crear_cliente(cliente: ClienteEntrada):
    global id_counter
    nuevo = ClienteSalida(id=id_counter, **cliente.dict())  # Convertimos entrada a salida con ID
    clientes_db.append(nuevo)       # Lo guardamos en la lista
    id_counter += 1                 # Aumentamos el contador
    return nuevo                    # Devolvemos el cliente creado

# Ruta para listar todos los clientes
@router.get("/clientes", response_model=list[ClienteSalida])
def listar_clientes():
    return clientes_db              # Retorna la lista completa

# Ruta para obtener un cliente por ID
@router.get("/clientes/{id}", response_model=ClienteSalida)
def obtener_cliente(id: int):
    for c in clientes_db:
        if c.id == id:
            return c                # Si lo encuentra, lo devuelve
    raise HTTPException(404, detail="Cliente no encontrado")  # Si no, error 404

# Ruta para actualizar un cliente
@router.put("/clientes/{id}", response_model=ClienteSalida)
def actualizar_cliente(id: int, datos: ClienteEntrada):
    for i, c in enumerate(clientes_db):
        if c.id == id:
            actualizado = ClienteSalida(id=id, **datos.dict())  # Reemplaza el cliente existente
            clientes_db[i] = actualizado
            return actualizado
    raise HTTPException(404, detail="Cliente no encontrado")

# Ruta para eliminar un cliente
@router.delete("/clientes/{id}")
def eliminar_cliente(id: int):
    for i, c in enumerate(clientes_db):
        if c.id == id:
            del clientes_db[i]                        # Elimina el cliente por índice
            return {"mensaje": f"Cliente {id} eliminado"}
    raise HTTPException(404, detail="Cliente no encontrado")


---

###  `routers/tickets.py` – Lógica de tickets


In [ ]:
from fastapi import APIRouter, HTTPException
from models import TicketEntrada, TicketSalida
from routers.clientes import clientes_db  # Importamos los clientes para validar relaciones

router = APIRouter()
tickets_db = []              # Base de datos simulada de tickets
ticket_id_counter = 1        # Contador de tickets

# Ruta para crear un nuevo ticket
@router.post("/tickets", response_model=TicketSalida, status_code=201)
def crear_ticket(ticket: TicketEntrada):
    global ticket_id_counter

    # Verificamos que el cliente exista
    if not any(c.id == ticket.cliente_id for c in clientes_db):
        raise HTTPException(400, detail="Cliente no válido")

    # Creamos el ticket con estado "Abierto"
    nuevo = TicketSalida(id=ticket_id_counter, estado="Abierto", **ticket.dict())
    tickets_db.append(nuevo)
    ticket_id_counter += 1
    return nuevo

# Ruta para listar todos los tickets
@router.get("/tickets", response_model=list[TicketSalida])
def listar_tickets():
    return tickets_db

# Ruta para obtener un ticket por ID
@router.get("/tickets/{id}", response_model=TicketSalida)
def obtener_ticket(id: int):
    for t in tickets_db:
        if t.id == id:
            return t
    raise HTTPException(404, detail="Ticket no encontrado")

# Ruta para actualizar un ticket existente
@router.put("/tickets/{id}", response_model=TicketSalida)
def actualizar_ticket(id: int, datos: TicketEntrada):
    for i, t in enumerate(tickets_db):
        if t.id == id:
            if not any(c.id == datos.cliente_id for c in clientes_db):
                raise HTTPException(400, detail="Cliente no válido")
            actualizado = TicketSalida(id=id, estado="Actualizado", **datos.dict())
            tickets_db[i] = actualizado
            return actualizado
    raise HTTPException(404, detail="Ticket no encontrado")

# Ruta para eliminar un ticket
@router.delete("/tickets/{id}")
def eliminar_ticket(id: int):
    for i, t in enumerate(tickets_db):
        if t.id == id:
            del tickets_db[i]
            return {"mensaje": f"Ticket {id} eliminado"}
    raise HTTPException(404, detail="Ticket no encontrado")


---

###  `main.py` – Entrada principal


In [ ]:
from fastapi import FastAPI
from routers import clientes, tickets  # Importamos routers de clientes y tickets

# Creamos la instancia de la aplicación FastAPI
app = FastAPI(
    title="API Empresarial – Soporte Técnico",
    version="1.0"
)

# Ruta raíz, útil como test inicial
@app.get("/")
def raiz():
    return {"mensaje": "Bienvenido a la API de Soporte Técnico Empresarial"}

# Registramos los routers modularmente
app.include_router(clientes.router, tags=["Clientes"])
app.include_router(tickets.router, tags=["Tickets"])


---

### ✅ ¿Cómo ejecutar este proyecto?

1. Instala dependencias:

```bash
pip install -r requirements.txt
```

2. Ejecuta el servidor:

```bash
uvicorn main:app --reload
```

3. Abre el navegador y visita:
   [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs)

---


###  `requirements.txt` – Dependencias

```txt
fastapi==0.110.0         # Framework principal para la API
uvicorn[standard]==0.29.0  # Servidor ASGI para desarrollo
pydantic[email] 
```


## **Paso a paso completo** para probar todas las APIs del proyecto usando:

*  **FastAPI Docs** (`Swagger UI`)
*  **ReDoc**
*  **Postman**

---

##  1. Iniciar el servidor

Abre una terminal y navega al directorio donde está tu archivo `main.py`. Luego ejecuta:

```bash
uvicorn main:app --reload
```

Esto arrancará el servidor en:

```
http://127.0.0.1:8000
```

---


##  2. Usar FastAPI Docs (`Swagger UI`)

###  Accede desde tu navegador:

```
http://127.0.0.1:8000/docs
```

Allí podrás:

* Ver todas las rutas agrupadas por "Clientes" y "Tickets".
* Probar cada endpoint directamente sin usar otra herramienta.

### ✔ Pruebas recomendadas en Swagger:

1. `POST /clientes`: Crea un nuevo cliente
   **Body de ejemplo:**



In [ ]:
   {
     "nombre": "Carlos Ruiz",
     "email": "carlos@empresa.com"
   }


2. `GET /clientes`: Muestra todos los clientes

3. `GET /clientes/{id}`: Consulta un cliente por ID

4. `PUT /clientes/{id}`: Actualiza un cliente existente
   **Body actualizado:**



In [ ]:
   {
     "nombre": "Carlos R. Ruiz",
     "email": "cruiz@empresa.com"
   }


5. `DELETE /clientes/{id}`: Elimina un cliente

---

6. `POST /tickets`: Crea un ticket (usa un `cliente_id` válido)
   **Body:**



In [ ]:
   {
     "asunto": "Problema con software",
     "descripcion": "No puedo acceder al CRM desde esta mañana.",
     "cliente_id": 1
   }


7. `GET /tickets`: Lista todos los tickets

8. `GET /tickets/{id}`: Consulta un ticket por ID

9. `PUT /tickets/{id}`: Actualiza un ticket

10. `DELETE /tickets/{id}`: Elimina un ticket

---


## 3. Usar ReDoc

ReDoc es otra vista de documentación, más descriptiva.

###  Accede desde tu navegador:

```
http://127.0.0.1:8000/redoc
```

* No puedes hacer pruebas interactivas aquí, pero la documentación es más detallada para usuarios técnicos o clientes internos.

---


##  4. Usar Postman

###  1. Crear una colección en Postman

Puedes crear una colección llamada `API CRM`.

###  2. Crear y probar endpoints:

Aquí tienes los ejemplos listos para copiar en Postman:

####  Crear cliente (POST)

* URL: `http://127.0.0.1:8000/clientes`
* Method: `POST`
* Body → `raw` → `JSON`:



In [ ]:
{
  "nombre": "Laura Torres",
  "email": "laura@empresa.com"
}


####  Ver todos los clientes (GET)

* URL: `http://127.0.0.1:8000/clientes`
* Method: `GET`

#### Consultar cliente por ID (GET)

* URL: `http://127.0.0.1:8000/clientes/1`
* Method: `GET`

####  Actualizar cliente (PUT)

* URL: `http://127.0.0.1:8000/clientes/1`
* Method: `PUT`
* Body:



In [ ]:
{
  "nombre": "Laura T. Torres",
  "email": "ltorres@empresa.com"
}


####  Eliminar cliente (DELETE)

* URL: `http://127.0.0.1:8000/clientes/1`
* Method: `DELETE`

---


####  Crear ticket (POST)

* URL: `http://127.0.0.1:8000/tickets`
* Method: `POST`
* Body:



In [ ]:
{
  "asunto": "Error en factura",
  "descripcion": "Se generó mal el monto en la factura de abril.",
  "cliente_id": 2
}


####  Ver todos los tickets (GET)

* URL: `http://127.0.0.1:8000/tickets`
* Method: `GET`

####  Consultar ticket (GET)

* URL: `http://127.0.0.1:8000/tickets/1`
* Method: `GET`

####  Actualizar ticket (PUT)

* URL: `http://127.0.0.1:8000/tickets/1`
* Method: `PUT`
* Body:



In [ ]:
{
  "asunto": "Error corregido en factura",
  "descripcion": "Ya fue corregido el monto pero falta reenviar factura.",
  "cliente_id": 2
}


####  Eliminar ticket (DELETE)

* URL: `http://127.0.0.1:8000/tickets/1`
* Method: `DELETE`

---


## **Caso Real: API de Gestión de Inventarios**

**Descripción del Proyecto:**
Imaginemos que tenemos una tienda que vende productos en línea y necesita gestionar su inventario. Queremos crear una API que permita a los administradores gestionar productos, registrar nuevas entradas de productos, consultar el estado del inventario y realizar ventas. La API también debe permitir realizar validaciones y devolver respuestas estructuradas.

---


### **Requisitos del Proyecto**

1. **Tecnologías a Usar:**

   * FastAPI para crear la API.
   * Pydantic para la validación de datos.
   * Uvicorn para ejecutar el servidor.
   * SQLAlchemy o cualquier base de datos en memoria (SQLite) para almacenar los productos.

2. **Modelos y Rutas:**

   * **Producto:** Representa un producto en el inventario (nombre, descripción, precio, stock).
   * **Venta:** Representa una venta, que reduce el stock de los productos.
   * **Rutas:** Crear un producto, mostrar todos los productos, realizar una venta, consultar el inventario, etc.

3. **Validaciones y Respuestas:**

   * Validación de entradas con Pydantic.
   * Respuestas estructuradas con datos limpios.

---


### **Estructura del Proyecto**

```
/mi_proyecto
│
├── /routers
│   └── productos.py       # Archivo de rutas de productos
├── crud.py               # Lógica de las funciones CRUD
├── main.py               # Archivo principal para ejecutar FastAPI
├── models.py             # Modelos de datos (Pydantic)
└── requirements.txt      # Dependencias
```
---

### **Paso 1: Crear `models.py`**

En este archivo definimos los modelos con Pydantic.


In [ ]:
from pydantic import BaseModel
from typing import List

# Modelo para la entrada de un producto (para la creación de nuevos productos)
class ProductoEntrada(BaseModel):
    nombre: str
    descripcion: str
    precio: float
    stock: int

# Modelo para la salida de un producto (para las respuestas de la API)
class ProductoSalida(BaseModel):
    id: int
    nombre: str
    precio: float

    model_config = {
        "from_attributes": True
    }


# Modelo para la venta (cuando se realiza una venta)
class Venta(BaseModel):
    producto_id: int
    cantidad: int


### **Paso 2: Crear `crud.py`**

Aquí se maneja la lógica de negocio, como la creación de productos, el registro de ventas, y la consulta de inventarios.



In [ ]:
from fastapi import APIRouter, HTTPException
from typing import List
from crud import crear_producto, obtener_productos, obtener_producto_por_id, actualizar_producto, eliminar_producto, realizar_venta
from models import ProductoEntrada, ProductoSalida, Venta

router = APIRouter()

# Crear un producto nuevo
@router.post("/productos", response_model=ProductoSalida)
def crear_producto_api(producto: ProductoEntrada):
    try:
        return crear_producto(producto)
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

# Obtener todos los productos
@router.get("/productos", response_model=List[ProductoSalida])
def obtener_productos_api():
    return obtener_productos()

# Obtener un producto por ID
@router.get("/productos/{producto_id}", response_model=ProductoSalida)
def obtener_producto_por_id_api(producto_id: int):
    producto = obtener_producto_por_id(producto_id)
    if producto is None:
        raise HTTPException(status_code=404, detail="Producto no encontrado")
    return producto

# Actualizar un producto
@router.put("/productos/{producto_id}", response_model=ProductoSalida)
def actualizar_producto_api(producto_id: int, producto: ProductoEntrada):
    producto_actualizado = actualizar_producto(producto_id, producto)
    if producto_actualizado is None:
        raise HTTPException(status_code=404, detail="Producto no encontrado")
    return producto_actualizado

# Eliminar un producto
@router.delete("/productos/{producto_id}", response_model=ProductoSalida)
def eliminar_producto_api(producto_id: int):
    producto_eliminado = eliminar_producto(producto_id)
    if producto_eliminado is None:
        raise HTTPException(status_code=404, detail="Producto no encontrado")
    return producto_eliminado

# Realizar una venta
@router.post("/ventas", response_model=ProductoSalida)
def realizar_venta_api(venta: Venta):
    try:
        return realizar_venta(venta)
    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))


### **Paso 3: Crear `productos.py` (Router)**

Definimos las rutas para las operaciones de inventario.



In [ ]:
from fastapi import APIRouter, HTTPException
from typing import List
from crud import crear_producto, obtener_productos, obtener_producto_por_id, actualizar_producto, eliminar_producto
from models import ProductoEntrada, ProductoSalida, Venta

router = APIRouter()

# Crear un producto nuevo
@router.post("/productos", response_model=ProductoSalida)
def crear_producto_api(producto: ProductoEntrada):
    try:
        return crear_producto(producto)
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

# Obtener todos los productos
@router.get("/productos", response_model=List[ProductoSalida])
def obtener_productos_api():
    return obtener_productos()

# Obtener un producto por ID
@router.get("/productos/{producto_id}", response_model=ProductoSalida)
def obtener_producto_por_id_api(producto_id: int):
    producto = obtener_producto_por_id(producto_id)
    if producto is None:
        raise HTTPException(status_code=404, detail="Producto no encontrado")
    return producto

# Actualizar un producto
@router.put("/productos/{producto_id}", response_model=ProductoSalida)
def actualizar_producto_api(producto_id: int, producto: ProductoEntrada):
    producto_actualizado = actualizar_producto(producto_id, producto)
    if producto_actualizado is None:
        raise HTTPException(status_code=404, detail="Producto no encontrado")
    return producto_actualizado

# Eliminar un producto
@router.delete("/productos/{producto_id}", response_model=ProductoSalida)
def eliminar_producto_api(producto_id: int):
    producto_eliminado = eliminar_producto(producto_id)
    if producto_eliminado is None:
        raise HTTPException(status_code=404, detail="Producto no encontrado")
    return producto_eliminado

# Realizar una venta
@router.post("/ventas", response_model=ProductoSalida)
def realizar_venta_api(venta: Venta):
    try:
        return realizar_venta(venta)
    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))


### **Paso 4: Crear `main.py`**

Este archivo integra todo, inicializando la API con FastAPI.



In [ ]:
from fastapi import FastAPI
from routers import productos  # Importa el módulo, no el router directamente

app = FastAPI(
    title="API Empresarial – Gestión de Productos y Ventas",
    version="1.0"
)

@app.get("/")
def raiz():
    return {"mensaje": "Bienvenido a la API Empresarial – Gestión de Productos y Ventas"}

# Asegúrate de que productos.router esté bien referenciado
app.include_router(productos.router, tags=["Productos"])


### **Paso 5: Crear `requirements.txt`**

Asegúrate de incluir todas las dependencias necesarias en el archivo `requirements.txt`.

```plaintext
fastapi
uvicorn
pydantic
```



### **Paso 6: Ejecutar el Proyecto**

1. **Instalar dependencias:**

   Si no tienes las dependencias instaladas, ejecuta:

   ```bash
   pip install -r requirements.txt
   ```

2. **Correr el servidor:**

   Para ejecutar el servidor, usa el siguiente comando:

   ```bash
   uvicorn main:app --reload
   ```



## **Paso a paso completo para probar todas las API con Swagger Docs** (`/docs`) de tu proyecto FastAPI:

---


###  1. **Asegúrate de que el servidor esté corriendo**

Desde la terminal, estando en la carpeta raíz del proyecto, ejecuta:

```bash
uvicorn main:app --reload
```

* `main` es el nombre del archivo (`main.py`)
* `app` es la instancia de FastAPI creada dentro de ese archivo
* `--reload` reinicia el servidor automáticamente cuando detecta cambios

---

###  2. **Abre el navegador y accede a Swagger UI**

Dirígete a:

```
http://127.0.0.1:8000/docs
```

Allí verás la documentación **interactiva** generada automáticamente con FastAPI.

---

###  3. **Probar las rutas de productos**

Supongamos que tus rutas están organizadas bajo `/productos` y `/ventas`, y que ya están incluidas en `main.py`.

####  **POST /productos** – Crear producto

* Da clic en `POST /productos`
* Haz clic en **"Try it out"**
* Llena el cuerpo de la solicitud con un ejemplo como:

```json
{
  "nombre": "Teclado mecánico",
  "descripcion": "Teclado retroiluminado con switches azules",
  "precio": 199.99,
  "stock": 25
}
```

* Presiona **Execute**
* Verás la respuesta del servidor y el código HTTP

---

####  **GET /productos** – Obtener todos los productos

* Haz clic en `GET /productos`
* Presiona **Try it out** → **Execute**
* Verás la lista de productos almacenados

---

####  **GET /productos/{producto\_id}** – Consultar producto por ID

* Haz clic en esta ruta
* Presiona **Try it out**
* Ingresa un `producto_id` (ej: `1`)
* Presiona **Execute**

---

####  **PUT /productos/{producto\_id}** – Actualizar producto

* Haz clic en esta ruta
* Try it out
* Ingresa el `producto_id` (ej: `1`)
* En el body:

```json
{
  "nombre": "Teclado manual-mecanico",
  "descripcion": "Teclado retroiluminado con switches azules",
  "precio": 199.99,
  "stock": 25
}
```

* Execute

---

####  **DELETE /productos/{producto\_id}** – Eliminar producto

* Haz clic en esta ruta
* Try it out
* Ingresa el ID del producto a eliminar
* Execute

---

###  4. **Probar ventas**

####  **POST /ventas** – Realizar venta

* Clic en `POST /ventas`
* Try it out
* Body:

```json
{
  "producto_id": 1,
  "cantidad": 2
}
```

* Execute

---



##  Recursos Complementarios

* [https://docs.pydantic.dev](https://docs.pydantic.dev)
* [https://fastapi.tiangolo.com/tutorial/body/](https://fastapi.tiangolo.com/tutorial/body/)
* [https://docs.pydantic.dev/latest/usage/validators/](https://docs.pydantic.dev/latest/usage/validators/)
* [https://cursor.so](https://cursor.so) (IA para refactor y modelos)

---
